In [ ]:
def modelfit1(dtrain, dtest, predictors,useTrainCV=True, thresh = .8, cv_folds=5, early_stopping_rounds=50):
    
    alg = XGBClassifier(
        learning_rate =lr,
        n_estimators=ne,
        max_depth=md,
        min_child_weight=mcw,
        gamma=g,
        subsample=ss,
        colsample_bytree=cst,
        objective= obj,
        nthread=nt,
        scale_pos_weight=spw,
        seed=seed)
    
    print ("Parameter Tuning")
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        print ("Current Paramter:\n", xgb_param)
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics = 'auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        print (cvresult)
        ne_new = n_estimators=cvresult.shape[0]
        alg.set_params(n_estimators=cvresult.shape[0])
        print ("\nNew paramter\n", alg.get_xgb_params())
        
    #Fit the algorithm on the data
    print ("Fitting Algorithm")
    eval_set = [(dtrain[predictors],dtrain[target]), (dtest[predictors],dtest[target])]
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc', eval_set = eval_set, 
            verbose = True, early_stopping_rounds=early_stopping_rounds) #scoring_xg
        
    #Predict training set:
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    dtrain_predictions = (dtrain_predprob > thresh).astype(int)
            
    #Print training data report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % (metrics.accuracy_score(dtrain[target].values, dtrain_predictions)*100))
    print ("AUC Score (Train): %f" % (metrics.roc_auc_score(dtrain[target], dtrain_predprob)))
    print ("F1 score (Train): %f" % (metrics.f1_score(dtrain[target], dtrain_predictions)*100))
    print ("Precision score (Train): %f" % (metrics.precision_score(dtrain[target], dtrain_predictions)*100))
    print ("Recall score (Train): %f" % (metrics.recall_score(dtrain[target], dtrain_predictions)*100))
    
    #Predict on testing data:
    results = dtest.copy()
    results['predprob'] = alg.predict_proba(results[predictors])[:,1]
    results['predict'] = (results['predprob'] > thresh).astype(int)
    
    #Print validation data report:
    print ('\n AUC Score (Valid): %f' % (metrics.roc_auc_score(results[target], results['predprob'])))
    print ("F1 score (Valid): %f" % (metrics.f1_score(results[target], results['predict'])*100))
    print ("Precision score (Valid): %f" % (metrics.precision_score(results[target], results['predict'])*100))
    print ("Recall score (Valid): %f" % (metrics.recall_score(results[target], results['predict'])*100))
    
    # Plot feature importance
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances', figsize = (20, 5))
    plt.ylabel('Feature Importance Score')
    
    # retrieve performance metrics
    results = alg.evals_result()
    epochs = len(results['validation_0']['auc'])
    x_axis = range(0, epochs)
    
    # plot auc
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['auc'], label='Train')
    ax.plot(x_axis, results['validation_1']['auc'], label='Test')
    ax.legend()
    plt.ylabel('auc')
#     plt.title()
    return alg#, ne_new

In [3]:
import pandas as pd
import os
import numpy as np
import time
pd.set_option('mode.chained_assignment', 'raise')

import requests
import json

from collections import Counter
import itertools as itool

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import  metrics
# from sklearn.model_selection import GridSearchCV

import datetime
from io import StringIO
from sklearn.externals import joblib

import matplotlib.pyplot as plt
import re
from datetime import timedelta

%matplotlib inline

print('using my_model_func-ver3.1')

using my_model_func-ver2+


In [ ]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

# Fetch Market List

In [ ]:
def fetch_market_list(status = 'active'):
    headers = {
   'Origin': 'https://app.prediction.exchange/',
   'Accept-Encoding': 'gzip, deflate',
   'Accept-Language': 'en-US,en;q=0.8',
   'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
   'Content-Type': 'application/json;charset=UTF-8',
   'Accept': 'application/json, text/plain, */*',
   'Referer': 'https://app.prediction.exchange/csv-download',
   'Connection': 'keep-alive',
    }
    
    r = requests.get('https://app.prediction.exchange/api/btrx/market', headers=headers)
    active_curr_list = json.loads(r.text)['response']
    
    r = requests.get('https://app.prediction.exchange/api/btrx/market/inactive', headers=headers)
    all_curr_list = json.loads(r.text)['response']
    all_curr_list = [d['name'] for d in all_curr_list]
    
    inactive_curr_list = [c for c in all_curr_list if c not in active_curr_list]
    
    if status == 'active':
        return active_curr_list
    elif status == 'all':
        return all_curr_list
    elif status == 'inactive':
        return inactive_curr_list

# Input data as csv

In [ ]:
def load_data_csv_2(start_date, end_date, data_type):
   
    date_range = pd.date_range(start=start_date, end=data_date, freq='D')
    folder_name_list = list(set([(100*(x.year%2000) + x.month) for x in date_range]))

    if data_type == 'trade':
        data_path = '/home/ambuj/rolled-up_data_new'
    elif data_type == 'order':
        data_path = '/home/ambuj/order_data_new'

    final_data_file_created = 0
    final_data = pd.DataFrame()
    for fold_ind, folder in enumerate(folder_name_list):
        final_data_path = os.path.join(data_path, str(folder))
        trade_file_name = os.path.join(final_data_path, list_d[list_i]+'.csv')

        if os.path.isfile(trade_file_name):
            data = fetch_data_csv(final_data_path, list_d[list_i]+'.csv', data_type)
            if data.empty: ## Confirm if this is to be removed
    #                 print ("Data Empty", folder)
               continue
        else:
            continue

        if final_data_file_created == 0:
            final_data = pd.DataFrame(columns = data.columns)
            for col in final_data.columns:
                final_data[col] = final_data[col].astype(data[col].dtype)
            final_data_file_created = 1

        final_data = pd.concat([final_data, data])

    return final_data

In [ ]:
## Laod the all data at once in dictionary

def load_data(data_path, curr, folder_name_list, data_type):
   
    print(curr,data_type)
    
    final_data_created = 0
    final_data = pd.DataFrame()
    for fold_ind, folder in enumerate(folder_name_list):
        
#         print(folder)
        
        final_data_path = os.path.join(data_path, str(folder))
        
#         print(final_data_path)
        
        data_file_name = os.path.join(final_data_path, curr+'.csv')

        if os.path.isfile(data_file_name):
            data = fetch_data_csv(final_data_path, curr+'.csv', data_type) 
#             print(data.shape)

            if data.empty: ## Confirm if this is to be removed
                print ("Data Empty", folder)
                continue
        else:
            print ("File does not exist!", folder)
            continue

        if final_data_created == 0:
            final_data = pd.DataFrame(columns = data.columns)
            for col in final_data.columns:
                final_data[col] = final_data[col].astype(data[col].dtype)
            final_data_created = 1

        final_data = pd.concat([final_data, data])
#         print(final_data.shape)
   
    return final_data

In [ ]:
def fetch_data_csv(path, market, data_type):
    
    if data_type == 'trade':
        try:
            data = pd.read_csv(os.path.join(path,market), index_col=['Timestamp'], parse_dates=['Timestamp'])
        except:
            data = pd.DataFrame()
    elif data_type == 'order':
        try:
            data = pd.read_csv(os.path.join(path,market), index_col=['timestamp'], parse_dates=['timestamp'], low_memory = False)
        except:
            data = pd.DataFrame()
    
    return data

# Data preprocessing

In [ ]:
# Filling NA values

def fill_na(df, vol_win = 48*60):
    
    # To get the data at unrecorded timestamps
    df_temp = df.resample('T').mean()
    
    # to fill missing Currency Pair values
    if not df_temp.empty:
        df_temp.loc[:, 'Curreny_Pair'] = df.at[df.index[0],'Curreny_Pair']
#         df_temp.loc[:, '_id'] = df.at[df.index[0],'_id']
    
    # To fill volumes and no. of transaction by last 48 hrs average
#     for col in ['Volume_first_currency','Volume_second_currency' ,'No_of_transaction' ]:
#         df_temp[col] = df[col].fillna(pd.rolling_mean(df[col], vol_win, min_periods=1))
    df_temp['Volume_first_currency'].fillna(df_temp['Volume_first_currency'].rolling(window = vol_win, min_periods = 1).mean(), inplace = True)
    df_temp['Volume_second_currency'].fillna(df_temp['Volume_second_currency'].rolling(window = vol_win, min_periods = 1).mean(), inplace = True)
    df_temp['No_of_transaction'].fillna(df_temp['No_of_transaction'].rolling(window = vol_win, min_periods = 1).mean(), inplace = True)

    # To fill Prices by forward values
#     for col in ['Open', 'High', 'Low', 'Close', 'Weighted_price']:
    df_temp['Open'].fillna(method='ffill', inplace=True)
    df_temp['High'].fillna(method='ffill', inplace=True)
    df_temp['Low'].fillna(method='ffill', inplace=True)
    df_temp['Close'].fillna(method='ffill', inplace=True)
    df_temp['Weighted_price'].fillna(method='ffill', inplace=True)
        
#     # To drop initial rows with na values
#     df_temp.dropna(subset=['Open'],inplace=True)
    
    return df_temp

# Trade Features

In [ ]:
# RSI

def gain(x):
    if(pd.isnull(x)):
        return np.nan
    elif(x>0):
        return x
    else:
        return 0

def loss(x):
    if(pd.isnull(x)):
        return np.nan
    elif(x<0):
        return abs(x)
    else:
        return 0

def rsi(df, rsi_win_array, rolling_win):
    
    df_original = df.copy()
    df_temp = df.copy()
    d={}
    interval = 15

    df_temp['change'] = df_temp['Close'] - df_temp['Close'].shift(rolling_win)
    df_temp['gain'] = df_temp['change'].apply(lambda x: gain(x))
    df_temp['loss'] = df_temp['change'].apply(lambda x: loss(x))

        
    for rsi_win in rsi_win_array:
        
        temp = df_temp.copy()
        
        for i in range(int(rolling_win/interval)):

            temp['rs'] = temp.loc[::rolling_win, 'gain'].rolling(rsi_win).mean()/ \
                         temp.loc[::rolling_win,'loss'].rolling(rsi_win).mean()
        
            df_temp_1 = temp.dropna(subset=['rs'],axis=0, how='any')
            d["{}".format(i)] = df_temp_1
            temp.drop(temp.index[:interval], inplace = True)
    

        result =  pd.concat([ d['{}'.format(j)] for j in range(len(d)) ])
        result.sort_index(inplace=True)

        df_original['rs'] = result['rs']
        df_original['rsi'+str(rsi_win)] = 100 - (100/(1+df_original['rs']))
        df_original.drop('rs', axis = 1, inplace = True)

    return df_original

In [ ]:
# Volume Average

def volume_feat(data):
    temp = data.copy()
    
    temp['volume_acc_14_24_60'] = temp['Volume_first_currency'].rolling(14*24*60, min_periods = 12*24*60).sum()
    temp['volume_accu_168_60'] = temp['Volume_first_currency'].rolling(168*60, min_periods = 144*60).sum()
    temp['volume_accu_144_60'] = temp['Volume_first_currency'].rolling(144*60, min_periods = 120*60).sum()
    temp['volume_accu_120_60'] = temp['Volume_first_currency'].rolling(120*60, min_periods = 96*60).sum()
    temp['volume_accu_96_60'] = temp['Volume_first_currency'].rolling(96*60, min_periods = 72*60).sum()
    temp['volume_accu_72_60'] = temp['Volume_first_currency'].rolling(72*60, min_periods = 54*60).sum()
    temp['volume_accu_48_60'] = temp['Volume_first_currency'].rolling(48*60, min_periods = 36*60).sum()
    temp['volume_accu_24_60'] = temp['Volume_first_currency'].rolling(24*60, min_periods = 18*60).sum()
    temp['volume_accu_240'] = temp['Volume_first_currency'].rolling(240, min_periods = 200).sum()
    temp['volume_accu_120'] = temp['Volume_first_currency'].rolling(120, min_periods = 100).sum()
    temp['volume_accu_60'] = temp['Volume_first_currency'].rolling(60, min_periods = 50).sum()
    temp['volume_accu_30'] = temp['Volume_first_currency'].rolling(30, min_periods = 25).sum()
    temp['volume_accu_15'] = temp['Volume_first_currency'].rolling(15, min_periods = 13).sum()
    temp['volume_accu_10'] = temp['Volume_first_currency'].rolling(10, min_periods = 10).sum()
    temp.iloc[:, -13:] = (temp.iloc[:,-13:].div(temp.volume_acc_14_24_60, axis =0)).astype(float)
    
    return temp

In [ ]:
# Price Average

def price_mean_feat(data):
    temp = data.copy()
    
    temp['price_mean_14_24_60'] = temp['Weighted_price'].rolling(14*24*60, min_periods = 12*24*60).sum() #----> correct this
    temp['price_mean_168_60'] = temp['Weighted_price'].rolling(168*60, min_periods = 144*60).mean()
    temp['price_mean_144_60'] = temp['Weighted_price'].rolling(144*60, min_periods = 120*60).mean()
    temp['price_mean_120_60'] = temp['Weighted_price'].rolling(120*60, min_periods = 96*60).mean()
    temp['price_mean_96_60'] = temp['Weighted_price'].rolling(96*60, min_periods = 72*60).mean()
    temp['price_mean_72_60'] = temp['Weighted_price'].rolling(72*60, min_periods = 54*60).mean()
    temp['price_mean_48_60'] = temp['Weighted_price'].rolling(48*60, min_periods = 36*60).mean()
    temp['price_mean_24_60'] = temp['Weighted_price'].rolling(24*60, min_periods = 18*60).mean()
    temp['price_mean_240'] = temp['Weighted_price'].rolling(240, min_periods = 200).mean()
    temp['price_mean_120'] = temp['Weighted_price'].rolling(120, min_periods = 100).mean()
    temp['price_mean_60'] = temp['Weighted_price'].rolling(60, min_periods = 50).mean()
    temp['price_mean_30'] = temp['Weighted_price'].rolling(30, min_periods = 25).mean()
    temp['price_mean_15'] = temp['Weighted_price'].rolling(15, min_periods = 13).mean()
    temp['price_mean_10'] = temp['Weighted_price'].rolling(10, min_periods = 10).mean()
    temp['current_price'] = temp['Close']
    temp.iloc[:, -14:] = (temp.iloc[:,-14:].div(temp.price_mean_14_24_60, axis =0)).astype(float)
    
    return temp

In [ ]:
# William

def william(data, rolling_win = 14*24*60):
    
    temp = data.copy()
    
    highest_high = temp['High'].rolling(rolling_win).max()
    lowest_low = temp['Low'].rolling(rolling_win).min()
    temp['william'] = ( (highest_high - temp['Close']) / (highest_high - lowest_low) * -100 )
    
    return temp

In [ ]:
# stoch_osci

def stoch_osci(data, rolling_win = 14*24*60):
    
    temp = data.copy()
    
    highest_high = temp['High'].rolling(rolling_win).max()
    lowest_low = temp['Low'].rolling(rolling_win).min()
    temp['stoch_osci'] = ( (temp['Close']-lowest_low) / (highest_high - lowest_low) ) #* 100 ) ---> correct this
    
    return temp

# Order Features

In [ ]:
def order_book_feature_power(data, trade_feat):
    
    #Convert dtypes to float
    print(data.columns)
    data.iloc[:,2:] = data.iloc[:,2:].astype(np.float64)
#     print (data['powerImbalancePower4Buy5Percent'].dtype)
#     k = data['powerImbalancePower4Buy5Percent'].apply(pd.to_numeric, errors='coerce')
#     print (k.dtype)
    #Impute missing data
    data.update(data[list(data.columns[14:])].fillna(method='ffill'))
    
    #Power Imbalance
    data_feat = pd.DataFrame(index = data.index)
    
    data_feat['pow_imbal_2_5'] = data['powerImbalancePower2Buy5Percent'] - data['powerImbalancePower2Sell5Percent']
    data_feat['pow_imbal_2_10'] = data['powerImbalancePower2Buy10Percent'] - data['powerImbalancePower2Sell10Percent']
    data_feat['pow_imbal_2_15'] = data['powerImbalancePower2Buy15Percent'] - data['powerImbalancePower2Sell15Percent']
    data_feat['pow_imbal_2_20'] = data['powerImbalancePower2Buy20Percent'] - data['powerImbalancePower2Sell20Percent']
    data_feat['pow_imbal_2_25'] = data['powerImbalancePower2Buy25Percent'] - data['powerImbalancePower2Sell25Percent']
    data_feat['pow_imbal_2_all'] = data['powerImbalancePower2BuyAll'] - data['powerImbalancePower2SellAll']
    
    data_feat['pow_imbal_4_5'] = data['powerImbalancePower4Buy5Percent'] - data['powerImbalancePower4Sell5Percent']
    data_feat['pow_imbal_4_10'] = data['powerImbalancePower4Buy10Percent'] - data['powerImbalancePower4Sell10Percent']
    data_feat['pow_imbal_4_15'] = data['powerImbalancePower4Buy15Percent'] - data['powerImbalancePower4Sell15Percent']
    data_feat['pow_imbal_4_20'] = data['powerImbalancePower4Buy20Percent'] - data['powerImbalancePower4Sell20Percent']
    data_feat['pow_imbal_4_25'] = data['powerImbalancePower4Buy25Percent'] - data['powerImbalancePower4Sell25Percent']
    data_feat['pow_imbal_4_all'] = data['powerImbalancePower4BuyAll'] - data['powerImbalancePower4SellAll']
    
    data_feat['pow_imbal_8_5'] = data['powerImbalancePower8Buy5Percent'] - data['powerImbalancePower8Sell5Percent']
    data_feat['pow_imbal_8_10'] = data['powerImbalancePower8Buy10Percent'] - data['powerImbalancePower8Sell10Percent']
    data_feat['pow_imbal_8_15'] = data['powerImbalancePower8Buy15Percent'] - data['powerImbalancePower8Sell15Percent']
    data_feat['pow_imbal_8_20'] = data['powerImbalancePower8Buy20Percent'] - data['powerImbalancePower8Sell20Percent']
    data_feat['pow_imbal_8_25'] = data['powerImbalancePower8Buy25Percent'] - data['powerImbalancePower8Sell25Percent']
    data_feat['pow_imbal_8_all'] = data['powerImbalancePower8BuyAll'] - data['powerImbalancePower8SellAll']
    
    data_feat['pow_adj_2_5'] = data['powerAdjustedPower2Buy5Percent'] - data['powerAdjustedPower2Sell5Percent']
    data_feat['pow_adj_2_10'] = data['powerAdjustedPower2Buy10Percent'] - data['powerAdjustedPower2Sell10Percent']
    data_feat['pow_adj_2_15'] = data['powerAdjustedPower2Buy15Percent'] - data['powerAdjustedPower2Sell15Percent']
    data_feat['pow_adj_2_20'] = data['powerAdjustedPower2Buy20Percent'] - data['powerAdjustedPower2Sell20Percent']
    data_feat['pow_adj_2_25'] = data['powerAdjustedPower2Buy25Percent'] - data['powerAdjustedPower2Sell25Percent']
    data_feat['pow_adj_2_all'] = data['powerAdjustedPower2BuyAll'] - data['powerAdjustedPower2SellAll']
    
    data_feat['pow_adj_4_5'] = data['powerAdjustedPower4Buy5Percent'] - data['powerAdjustedPower4Sell5Percent']
    data_feat['pow_adj_4_10'] = data['powerAdjustedPower4Buy10Percent'] - data['powerAdjustedPower4Sell10Percent']
    data_feat['pow_adj_4_15'] = data['powerAdjustedPower4Buy15Percent'] - data['powerAdjustedPower4Sell15Percent']
    data_feat['pow_adj_4_20'] = data['powerAdjustedPower4Buy20Percent'] - data['powerAdjustedPower4Sell20Percent']
    data_feat['pow_adj_4_25'] = data['powerAdjustedPower4Buy25Percent'] - data['powerAdjustedPower4Sell25Percent']
    data_feat['pow_adj_4_all'] = data['powerAdjustedPower4BuyAll'] - data['powerAdjustedPower4SellAll']
    
    data_feat['pow_adj_8_5'] = data['powerAdjustedPower8Buy5Percent'] - data['powerAdjustedPower8Sell5Percent']
    data_feat['pow_adj_8_10'] = data['powerAdjustedPower8Buy10Percent'] - data['powerAdjustedPower8Sell10Percent']
    data_feat['pow_adj_8_15'] = data['powerAdjustedPower8Buy15Percent'] - data['powerAdjustedPower8Sell15Percent']
    data_feat['pow_adj_8_20'] = data['powerAdjustedPower8Buy20Percent'] - data['powerAdjustedPower8Sell20Percent']
    data_feat['pow_adj_8_25'] = data['powerAdjustedPower8Buy25Percent'] - data['powerAdjustedPower8Sell25Percent']
    data_feat['pow_adj_8_all'] = data['powerAdjustedPower8BuyAll'] - data['powerAdjustedPower8SellAll']
    
    out = pd.concat([data_feat, trade_feat], axis = 1)
    return out

In [ ]:
def order_book_feature_volume(data, trade_feat):
    
    #Convert dtypes to float
    data.iloc[:,2:] = data.iloc[:,2:].apply(pd.to_numeric, errors = 'coerce')
    
    
    #Impute missing data
    data.update(data[list(data.columns[14:])].fillna(method='ffill'))
    
    #Power Imbalance
    data_feat = pd.DataFrame(index = data.index)
    data_feat['volume_acc_14_24_60'] = trade_feat['Volume_second_currency'].rolling(14*24*60, min_periods = 13*24*60).sum()
    data_feat['order_vol_buy_5'] = data['orderVolumeBuySecondaryCurrency5Percent']
    data_feat['order_vol_buy_10'] = data['orderVolumeBuySecondaryCurrency10Percent']
    data_feat['order_vol_buy_15'] = data['orderVolumeBuySecondaryCurrency15Percent']
    data_feat['order_vol_buy_20'] = data['orderVolumeBuySecondaryCurrency20Percent']
    data_feat['order_vol_buy_25'] = data['orderVolumeBuySecondaryCurrency25Percent']
    data_feat['order_vol_buy_All'] = data['orderVolumeBuySecondaryCurrencyAll']
    
    data_feat['order_vol_sell_5'] = data['orderVolumeSellSecondaryCurrency5Percent']
    data_feat['order_vol_sell_10'] = data['orderVolumeSellSecondaryCurrency10Percent']
    data_feat['order_vol_sell_15'] = data['orderVolumeSellSecondaryCurrency15Percent']
    data_feat['order_vol_sell_20'] = data['orderVolumeSellSecondaryCurrency20Percent']
    data_feat['order_vol_sell_25'] = data['orderVolumeSellSecondaryCurrency25Percent']
    data_feat['order_vol_sell_All'] = data['orderVolumeSellSecondaryCurrencyAll']
    
    data_feat['order_vol_diff_5'] = data_feat['order_vol_buy_5'] - data_feat['order_vol_sell_5']
    data_feat['order_vol_diff_10'] = data_feat['order_vol_buy_10'] - data_feat['order_vol_sell_10']
    data_feat['order_vol_diff_15'] = data_feat['order_vol_buy_15'] - data_feat['order_vol_sell_15']
    data_feat['order_vol_diff_20'] = data_feat['order_vol_buy_20'] - data_feat['order_vol_sell_20']
    data_feat['order_vol_diff_25'] = data_feat['order_vol_buy_25'] - data_feat['order_vol_sell_25']
    data_feat['order_vol_diff_All'] = data_feat['order_vol_buy_All'] - data_feat['order_vol_sell_All']
    
    data_feat = data_feat.iloc[:, 1:].div(data_feat.iloc[:,0], axis = 0)
    
    out = pd.concat([trade_feat, data_feat], axis = 1)
#     out.dropna(inplace=True)
    
    return out

# Target Variable

In [2]:
# To calculate Jump value(target variable)

def calc_jump(df, jump, window):
    
    df_temp = df.copy()
    
    win_min = window*60
    
    df_temp['highest_high'] = df_temp.High[::-1].rolling(win_min).max()[::-1] #highest high in x hr taken

    df_temp['pct_change_{}_{}'.format(jump,window)] = (( df_temp['highest_high'] - df_temp['Open']) / df_temp['Open'] ) * 100
    df_temp['class_{}_{}'.format(jump,window)] = np.where(df_temp['pct_change_{}_{}'.format(jump,window)] >= jump, 1, 0) 
    
    # # To drop all the NA Values
#     df_temp.dropna(subset=['highest_high'],inplace=True)
    
    #     To drop highest high column
    df_temp.drop(['highest_high'], axis=1, inplace=True)
    
    return df_temp

# Train Test Splitting

In [ ]:
def split_pct(df,train_size,validation_size):
     
    delta =  (df.index[-1] - df.index[0]) 
    
    train_end = (df.index[0] + (train_size*delta))
    
    train = df[df.index < train_end]
    temp_df = df[df.index >= train_end]
    
    valid_end = (temp_df.index[0] + validation_size*delta)
    
    validation = df[(df.index > train_end) & (df.index < valid_end)]
    
    test = df[df.index >= valid_end]
    
    return train,validation,test

# Training

In [1]:
def thresh_analysis_2(model, input_d, actual, thresh_start, hours):

## Exdtension of thresh_analysis where we consider only 1 prediction in x hours for each currency
    input_d['class_{}_{}'.format(jump,window)] = input_d.loc[:,'class_{}_{}'.format(jump,window)].astype(int)
    input_d = input_d.iloc[::sample_int_test,:]
    
    col = ['thresh','precision', 'recall', 'predict_count', 'precision_sel', 'predict_count_sel', 'Unique_Currency']
    thresh_res = pd.DataFrame(columns = col)
    ind = 0
    
    for t in np.arange(thresh_start, 100, 1):
        t /= 100.0
        print ("Threshold: ", t)
        preds = model.predict_proba(input_d[feature])
        input_d['pred'] = (preds[:,1] >= t).astype(int)
#         print(len(input_d['pred']))
        #Calculating precison coonsidering prediction at the interval of 24 hours
        temp = input_d[input_d.pred == 1]
#         print(len(temp['pred']))
        curr_unique = temp['Curreny_Pair'].unique()
#         print(curr_unique)
        
        new_input_d = pd.DataFrame()
        for curr in curr_unique:
#             print ("Currency Pair: ", curr)
            temp1 = temp[temp.Curreny_Pair == curr][['Curreny_Pair','pred','class_{}_{}'.format(jump,window)]]
#             print(temp1)
            
            while len(temp1) != 0:
                curr_time = temp1.index[0]
                new_input_d = pd.concat([new_input_d, temp1[temp1.index == curr_time]])
                drop_time = curr_time + timedelta(hours = hours)
                temp1 = temp1[temp1.index >= drop_time]
        
        if len(new_input_d) == 0:
            break
        
        thresh_res.loc[ind, col] = int(t*100), metrics.precision_score(input_d[actual], input_d['pred'])*100, \
                                metrics.recall_score(input_d[actual], input_d['pred'])*100, sum(input_d['pred']), \
                                (new_input_d['class_{}_{}'.format(jump,window)].sum()/float(len(new_input_d)))*100, len(new_input_d), len(curr_unique)
        ind = ind + 1
        
    ## Calculating best threshold and corresposnding precison
    prev_prec = 0
    cut_off_thresh = 0
    for ind, row in thresh_res.iterrows():
        if row['precision'] > prev_prec:
            cut_off_thresh = row['thresh']
            prev_prec = row['precision']
        else:
            break

    print ("Best Threshold and corresponding select precision: ", cut_off_thresh, thresh_res.at[ind-1, 'precision_sel'])
    return thresh_res, cut_off_thresh

In [3]:
def daily_precision(input_d, y, hours):
    
    res_col = ['predictions'] + y
    
    temp = input_d[input_d.predictions == 1]
    daily_results_raw = temp.resample('D').sum()
    for col in y:
        daily_results_raw['precision_raw_{}'.format(col)] = (daily_results_raw[col]/daily_results_raw.predictions)*100
        
    daily_results_raw.rename(columns = {'predictions': 'predictions_raw'}, inplace = True)
    
    curr_unique = temp['Curreny_Pair'].unique()
    new_input_d = pd.DataFrame()
    for curr in curr_unique:
#             print ("Currency Pair: ", curr)
        temp1 = temp[temp.Curreny_Pair == curr]#[['Currency_Pair','predictions','class_20']]
    
        while len(temp1) != 0:
            curr_time = temp1.index[0]
            new_input_d = pd.concat([new_input_d, temp1[temp1.index == curr_time]])
            drop_time = curr_time + timedelta(hours = hours)
            temp1 = temp1[temp1.index >= drop_time]
    
    
    daily_res = new_input_d.resample('D').sum()
    
    for col in y:
        daily_res['precision_sel_{}'.format(col)] = (daily_res[col]/daily_res.predictions)*100
    
    
    daily_res.rename(columns = {'predictions': 'predictions_sel'}, inplace = True)
    
    output = pd.merge(daily_results_raw, daily_res, left_index = True, right_index = True)
 
    output_summary = pd.DataFrame(columns = ['class_type', 'raw', 'sel']) ## Summary output dataframe
    
    for out_ind, col in enumerate(y):
        overall_raw_precision = (float(output[col+'_x'].sum())/float(output['predictions_raw'].sum()))*100
        overall_sel_precision = (float(output[col+'_y'].sum())/float(output['predictions_sel'].sum()))*100
        output_summary.at[out_ind, ['class_type','raw', 'sel']] = 'precision_'+col, overall_raw_precision, overall_sel_precision
    output_summary.at[out_ind+1, ['class_type','raw', 'sel']] = 'Total_predictions_count' \
                                , output['predictions_raw'].sum(), output['predictions_sel'].sum()
    
    output_col_raw, output_col_sel = [],[]
    
    for col in y:
        output_col_raw.append('precision_raw_'+col)
        output_col_sel.append('precision_sel_'+col)
    
    output_col_raw.append('predictions_raw')
    output_col_sel.append('predictions_sel')
    
    output_raw = output[output_col_raw]
    output_sel = output[output_col_sel]
    
    return output_summary, output_raw, output_sel

In [1]:
def my_plot_importance(booster, figsize, **kwargs): 
    from matplotlib import pyplot as plt
    from xgboost import plot_importance
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax, **kwargs)